# Lista 3

In [3]:
let polynomial1 = [1.0; 0.; -1.; 2.]

val polynomial1 : float list = [1.; 0.; -1.; 2.]


## Zadanie 1

Załóżmy, że wielomiany o współczynnikach rzeczywistych są reprezentowane jako listy współczynników od najwyższej potęgi do najniższej, np. [1.;0.;-1.;2.] oznacza wielomian x³ - x + 2. Napisz funkcję, która dla zadanej listy reprezentującej wielomian i dla danego argumentu x typu float, obliczy wartość tego wielomianu w punkcie x w oparciu o schemat Hornera. Napisz tę funkcję w dwóch wersjach: raz za pomocą rekursji ogonowej, a następnie bez jawnego użycia rekursji, korzystając z odpowiedniej funkcji bibliotecznej modułu List.

In [4]:
let horner1 x l = 
    let rec aux acc x = function
        | []    -> acc
        | y::ys -> aux (acc *. x +. y) x ys
    in
        aux 0. x l

let horner2 x l = List.fold_left (fun a b -> a *. x +. b) 0. l

val horner1 : float -> float list -> float = <fun>


val horner2 : float -> float list -> float = <fun>


In [9]:
horner1 1.5 polynomial1;;
horner2 1.5 polynomial1

- : float = 3.875


- : float = 3.875


## Zadanie 2
Załóżmy, że zmieniamy reprezentację wielomianu tak, by współczynniki w liście były uszeregowane od najniższej potęgi do najwyższej (wtedy [1.;0.;-1.;2.] oznacza wielomian 2x³ - x² + 1). Zmodyfikuj obie funkcje z Zadania 1 tak, by poprawnie liczyły wartość wielomianu w tej reprezentacji (rekursja w rozwiązaniu z jawną rekursją nie musi być ogonowa).

In [10]:
let horner3 x l = 
    let rec aux x = function
        | []    -> 0.
        | y::ys -> y +. x *. (aux x ys)
    in
        aux x l

let horner4 x l = List.fold_right (fun a b -> a +. b *. x) l 0.

val horner3 : float -> float list -> float = <fun>


val horner4 : float -> float list -> float = <fun>


In [11]:
horner3 1.5 @@ List.rev polynomial1;;
horner4 1.5 @@ List.rev polynomial1

- : float = 3.875


- : float = 3.875


## Zadanie 3
Dla reprezentacji wielomianu z Zadania 2 napisz funkcję (ponownie w dwóch wersjach) obliczającą pochodną wielomianu (np. dla listy [1.;0.;-1.;2.] funkcja ta powinna utworzyć listę [0.;-2.;6.]).

In [48]:
let derivative1 l = 
    let rec aux n = function
        | []    -> []
        | x::xs -> x *. n :: (aux (n +. 1.) xs)
    in if l == [] then [] else aux 1. (List.tl l)
    
    
let derivative1' = function
    | []    -> []
    | _::xs -> let rec aux n = function
        | []    -> []
        | y::ys -> y *. n :: (aux (n +. 1.) ys)
        in aux 1. xs

let derivative2 = function
    | []    -> []
    | _::xs -> let aux a b = 
        let n = fst a and acc = snd a in
        (n +. 1., b *. n :: acc)
    in
        List.rev @@ snd @@ List.fold_left aux (1.0 , []) xs
        
let derivative2' = function
    | []    -> []
    | _::xs -> let aux a b = 
        let n = fst b and acc = snd b in
        (n -. 1., a *. n :: acc)
    in
        snd @@ List.fold_right aux xs (float_of_int @@ List.length xs, [])

val derivative1 : float list -> float list = <fun>


val derivative1' : float list -> float list = <fun>


val derivative2 : float list -> float list = <fun>


val derivative2' : float list -> float list = <fun>


In [49]:
derivative1' [1.;0.;-1.;2.];;
derivative2 [1.;0.;-1.;2.];;
derivative2' [1.;0.;-1.;2.];;

- : float list = [0.; -2.; 6.]


- : float list = [0.; -2.; 6.]


- : float list = [0.; -2.; 6.]


## Zadanie 4
Niech macierz kwadratowa n × n będzie reprezentowana wierszami jako lista list.

* Napisz funkcję sprawdzającą, czy dana lista jest poprawną reprezentacją macierzy kwadratowej.
* Napisz funkcję, która dla zadanej macierzy kwadratowej i liczby naturalnej n wyznacza n-tą kolumnę macierzy.
* Wykorzystaj funkcję z poprzedniego punktu do napisania funkcji transpozycji macierzy.
* Napisz funkcję zip, która dla danych dwóch list równej długości tworzy listę złożoną z par elementów obu list znajdujących się na tych samych pozycjach, np. zip [1.;2.;3.] ["a";"b";"c"] = [(1.,"a");(2.,"b");(3.,"c")].
* Korzystając z funkcji zip, napisz funkcję zipf, która dla danych dwóch list typów 'a list i 'b list i funkcji dwuargumentowej f typu 'a -> 'b -> 'c tworzy listę złożoną z wartości funkcji f na argumentach z obu list położonych na tych samych pozycjach, np. zipf ( +. ) [1.;2.;3.] [4.;5.;6.] = [5.;7.;9.].
* Wykorzystując funkcję zipf napisz funkcję mult_vec, która oblicza iloczyn zadanego wektora i zadanej macierzy, np. mult_vec [1.;2.] [[2.;0.];[4.;5.]] = [10.;10.].
* Korzystając z funkcji multvec napisz funkcję mnożenia dwóch macierzy kwadratowych tego samego rozmiaru.

Uwaga: Wykorzystaj funkcje biblioteczne operujące na listach tak by Twoje rozwiązania były zwięzłe.

In [83]:
let is_square m = 
    let n = List.length m in
    let dims = List.map List.length m in
    List.for_all ((==)n) dims

let nth_column n m = List.fold_right (fun row acc -> List.nth row (n-1) :: acc) m []

let transpose m = 
    let n = List.length m in
    snd @@ List.fold_right (fun row acc -> (fst acc - 1 , nth_column (fst acc) m :: snd acc) ) m (n,[])
    
let transpose' m = 
    List.rev @@ snd @@ List.fold_left (fun acc row -> (fst acc+1 , nth_column (fst acc) m :: snd acc )) (1,[]) m
    
let zip l1 l2 = 
    let rec aux = function
        | (x::xs,y::ys) -> (x,y) :: aux (xs,ys)
        | (_,_)         -> []
    in
        aux (l1,l2)
        
let zipWith f l1 l2 = List.map (fun p -> f (fst p) (snd p) ) (zip l1 l2)


let rec list_sum = function
    | [] -> 0
    | x::xs -> x + list_sum xs

let mult_vec v m = List.map list_sum (List.map (zipWith (fun a b -> a * b) v ) @@ transpose m) 

let mult_matrices v1 v2 = List.fold_right (fun vect acc -> mult_vec vect v2 :: acc ) v1 []

val is_square : 'a list list -> bool = <fun>


val nth_column : int -> 'a list list -> 'a list = <fun>


val transpose : 'a list list -> 'a list list = <fun>


val transpose' : 'a list list -> 'a list list = <fun>


val zip : 'a list -> 'b list -> ('a * 'b) list = <fun>


val zipWith : ('a -> 'b -> 'c) -> 'a list -> 'b list -> 'c list = <fun>


val list_sum : int list -> int = <fun>


val mult_vec : int list -> int list list -> int list = <fun>


val mult_matrices : int list list -> int list list -> int list list = <fun>


In [86]:
is_square [[1;2;3];[4;5;6];[7;8;9]];;
nth_column 1 [[1;2;3];[4;5;6];[7;8;9]];;
transpose [[1;2;3];[4;5;6];[7;8;9]];;
transpose' [[1;2;3];[4;5;6];[7;8;9]];;

zip [1;2;3;4] ["a";"b";"c";"d"];;
zipWith (+) [1;2;3;4] [4;3;2;1];;

mult_vec [1;2] [[2;0];[4;5]];;

mult_matrices [[1;2];[3;4]] [[5;6];[7;8]];;

- : bool = true


- : int list = [1; 4; 7]


- : int list list = [[1; 4; 7]; [2; 5; 8]; [3; 6; 9]]


- : int list list = [[1; 4; 7]; [2; 5; 8]; [3; 6; 9]]


- : (int * string) list = [(1, "a"); (2, "b"); (3, "c"); (4, "d")]


- : int list = [5; 5; 5; 5]


- : int list = [10; 10]


- : int list list = [[19; 22]; [43; 50]]


## Zadanie 5
Napisz funkcję, która dla zadanej permutacji elementów dowolnego typu 'a, na którym zdefiniowany jest pewien porządek liniowy, znajduje kolejną - w porządku leksykograficznym - permutację tych samych elementów, np. dla permutacji (1,2,4,3) funkcja powinna zwrócić permutację (1,3,2,4), a dla permutacji (a,c,b) wynikiem powinna być permutacja (b,a,c). W przypadku, w którym zadana permutacja jest największa, funkcja powinna zwracać permutację najmniejszą. Do reprezentowania permutacji użyj list, ale w porządku odwróconym, tj. w przykładzie powyżej funkcja dla argumentu [3;4;2;1] powinna dać odpowiedź [4;2;3;1], a dla ['b';'c';'a'] - odpowiedź ['c';'a';'b']. Wykorzystaj tę funkcję do napisania funkcji generującej wszystkie permutacje danej listy.

In [ ]:
(* let next_perm p =  *)

## Zadanie 6
Pewnego razu powiedziano dwóm logikom P i S, że wybrano dwie liczby naturalne x i y takie, że 1 < x < y oraz x+y < 100. Co więcej, S otrzymał informację o wartości x+y, a P o wartości x*y. Po chwili P i S odbyli następującą rozmowę:

>P: Nie potrafię powiedzieć jakie to liczby.

>S: Wiedziałem o tym.

>P: A to w takim razie już potrafię.

>S: Ja już też.

Napisz program, którego treścią będzie powyższy dialog, a wartością - liczby x i y.

P: Znaczy, że nie są to dwie liczby pierwsze